In [ ]:
import sys
import os

# Detect environment
try:
    import google.colab
    IN_COLAB = True
    print("Running in Google Colab")
except ImportError:
    IN_COLAB = False
    print("Running locally or in Binder")

# Install UppASD if in Colab
if IN_COLAB:
    BRANCH_NAME = "python26"
    
    print(f"Installing UppASD from GitHub (branch: {BRANCH_NAME})...")
    print("This will take approximately 2-3 minutes...")
    
    # Clone repository
    !git clone --branch {BRANCH_NAME} https://github.com/UppASD/UppASD.git /content/UppASD
    
    # Install (prefer OpenBLAS on Colab). Use non-editable install so Colab imports work.
    %cd /content/UppASD
    !BLA_VENDOR=OpenBLAS FC=gfortran pip install . --quiet
    # Return to content directory
    %cd /content

    print("✓ Installation complete!")
    print("✓ UppASD is ready to use")

## Cloud Environment Setup

This cell detects if you're running on Google Colab and installs UppASD if needed. It does nothing when running locally or on Binder (where UppASD is pre-installed).

# UppASD Interactive Setup
Create and run atomistic spin dynamics simulations by defining geometry and Hamiltonian programmatically.

This notebook demonstrates how to:
1. Define crystal structure (lattice vectors, basis atoms)
2. Set up magnetic moments and exchange interactions
3. Configure simulation parameters via Python API
4. Run relaxation and dynamics
5. Analyze results

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

%matplotlib inline

## 1. Define Crystal Structure
Set up lattice parameters and atomic basis for a simple ferromagnet (e.g., bcc Fe).

In [ ]:
# Lattice parameters
alat = 2.87  # Angstrom (Fe bcc lattice constant)
lattice_vectors = np.array([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0]
]) * alat

# Basis atoms (fractional coordinates)
basis_fractional = np.array([
    [0.0, 0.0, 0.0],  # corner atom
    [0.5, 0.5, 0.5]   # body-center atom
])

# System size (supercell repetitions)
n1, n2, n3 = 10, 10, 10
natom = len(basis_fractional) * n1 * n2 * n3

print(f"System: {natom} atoms in {n1}×{n2}×{n3} supercell")
print(f"Lattice constant: {alat} Å")
print(f"Basis atoms: {len(basis_fractional)}")

## 2. Generate Atomic Coordinates
Build full supercell by tiling the unit cell.

In [ ]:
# Generate coordinates using helper function
from uppasd import notebook as nb

# Note: This notebook uses single-type system, so basis has no type column
# Create a compatible basis with type=1
basis_with_types = np.column_stack([basis_fractional, np.ones(len(basis_fractional))])

coords, atom_types = nb.generate_supercell_coordinates(
    lattice_vectors, basis_with_types, n1, n2, n3, scale=alat
)

print(f"Generated {len(coords)} coordinates")
print(f"First 5 atoms (Cartesian):")
print(coords[:5])

## 3. Initialize Magnetic Moments
Set initial spin directions and magnitudes.

In [ ]:
# Initialize magnetic moments using helper
moment_magnitude = 2.2

# Create ferromagnetic configuration with perturbation
moments = nb.create_ferromagnetic_moments(natom, moment_magnitude=moment_magnitude)
moments = nb.perturb_moments(moments, perturbation_scale=0.01, seed=42, renormalize=True)

print(f"Moments shape: {moments.shape}")
print(f"Mean magnetization: {np.mean(moments, axis=0)}")
print(f"Mean |M|: {np.linalg.norm(moments, axis=1).mean():.3f} μB")

## 4. Define Exchange Interactions
Set up Heisenberg exchange couplings J_ij for nearest neighbors.

In [ ]:
# For bcc Fe, typical exchange: J1 ≈ 21 meV (nearest neighbors)
j_nn = 21.0  # meV

# Find nearest-neighbor pairs (within 1.1 * a_bcc/2)
nn_cutoff = 1.1 * alat * np.sqrt(3) / 2

def find_neighbors(coords, cutoff):
    """Find neighbor pairs within cutoff distance."""
    from scipy.spatial import cKDTree
    tree = cKDTree(coords)
    pairs = tree.query_pairs(cutoff)
    return list(pairs)

neighbor_pairs = find_neighbors(coords, nn_cutoff)
print(f"Found {len(neighbor_pairs)} neighbor pairs (cutoff={nn_cutoff:.2f} Å)")
print(f"Example pairs: {list(neighbor_pairs)[:5]}")

## 5. Write Input Files for UppASD
Since we're using the existing Fortran backend, we need to create minimal input files.

In [ ]:
# Create working directory
work_dir = Path('./interactive_sim')
work_dir.mkdir(exist_ok=True)

# Create simulation configuration using helper
config = nb.create_relaxation_protocol('mc_only', steps=1000, temp=100)

# Add structure information
config.update({
    'simid': 'notebook',
    'ncell': [n1, n2, n3],
    'cell': np.eye(3) * alat / alat,  # Normalized
    'alat': alat,
    'posfile': './posfile.dat',
    'momfile': './momfile.dat',
    'exchange': './jfile.dat',
    'do_prnstruct': 1,
    'Mensemble': 1,
    'Initmag': 3,
    'ip_mode': 'M',
    'ip_mcanneal': True,
    'ip_mcanneal_params': [1, 300],
})

# Write inpsd.dat using helper
nb.write_inpsd_file(work_dir / 'inpsd.dat', config)

print("✓ Created inpsd.dat with MC protocol")

In [ ]:
# Write posfile using helper
nb.write_posfile(work_dir / 'posfile.dat', basis_with_types)

print(f"✓ Created posfile.dat ({len(basis_with_types)} basis atoms)")

In [ ]:
# Write momfile using helper
nb.write_momfile(work_dir / 'momfile.dat', {1: moment_magnitude}, 
                 atom_types=np.ones(len(basis_with_types), dtype=int))

print(f"✓ Created momfile.dat (μ = {moment_magnitude} μB)")

In [ ]:
# Write jfile.dat (exchange interactions)
# For bcc: each atom has 8 nearest neighbors at distance sqrt(3)*a/2
with open(work_dir / 'jfile.dat', 'w') as f:
    f.write(f"{len(basis_fractional)}\n")
    for i in range(len(basis_fractional)):
        # bcc structure: 8 NN bonds
        f.write(f"{i+1}  8\n")  # atom, num_neighbors
        # NN vectors for bcc (in units of lattice constant)
        nn_vecs = [
            [0.5, 0.5, 0.5], [-0.5, -0.5, -0.5],
            [0.5, -0.5, 0.5], [-0.5, 0.5, -0.5],
            [0.5, 0.5, -0.5], [-0.5, -0.5, 0.5],
            [0.5, -0.5, -0.5], [-0.5, 0.5, 0.5]
        ]
        for vec in nn_vecs:
            # atom_j, i_x, i_y, i_z, J_ij
            other = 1 if i == 1 else 0  # switch between sublattices
            f.write(f"{other+1}  {int(vec[0])}  {int(vec[1])}  {int(vec[2])}  {j_nn:.6f}\n")

print(f"Created jfile.dat (J_NN = {j_nn} meV)")

## 6. Run Simulation via Python API
Use the Simulator class to run the simulation.

In [ ]:
import os
from pathlib import Path
# Ensure work_dir exists (fallback)
if 'work_dir' not in globals():
    work_dir = Path('./relax_sim')
    work_dir.mkdir(exist_ok=True)

original_dir = os.getcwd()
try:
    os.chdir(work_dir)
except Exception as e:
    print(f"Could not switch to work_dir {work_dir}: {e}")
    work_dir = Path(original_dir)
    os.chdir(original_dir)

try:
    from uppasd import Simulator
    from uppasd.fileio import UppASDReader
    
    with Simulator() as sim:
        print(f"Initialized: {sim.natom} atoms, {sim.mensemble} ensembles")
        
        # Relax to equilibrium
        print("\nRelaxing to equilibrium...")
        final_moments = sim.relax(mode='M', temperature=100, steps=500)
        
        # Get final energy
        energy = sim.energy
        print(f"\nFinal energy: {energy:.3f} (code units)")
        
        # Get magnetization
        mag = np.mean(final_moments, axis=(1, 2))
        mag_norm = np.linalg.norm(mag)
        print(f"Magnetization: {mag}")
        print(f"|M|: {mag_norm:.3f}")
        
        # Run measurement phase to write output files
        print("\nRunning measurement phase...")
        files = sim.measure()
        print(f"✓ Measurement complete. Output files: {files}")
    
    # Load results using UppASDReader
    try:
        simid = 'notebook'  # use short simid compatible with Fortran filenames
        reader = UppASDReader(simid=simid)
        avg_data = reader.read_averages()
        print(f"\n✓ Loaded averages: {len(avg_data.get('time', []))} points")
    except Exception as e:
        print(f"Note: Could not load output files: {e}")
except ImportError:
    print("\n⚠️  UppASD Python module not found!")
    print("Install with: pip install -e /path/to/UppASD")
    print("\nAlternatively, run from terminal:")
    print(f"  cd {work_dir}")
    print("  sd < inpsd.dat")
finally:
    os.chdir(original_dir)

## 7. Visualize Results
Plot spin configuration (projection onto xy-plane).

In [ ]:
# Get final moments
moments_final = final_moments[:, :, 0]  # (3, natom)

# Sample subset for visualization (too many to plot)
sample_idx = np.random.choice(sim.natom, size=min(200, sim.natom), replace=False)
coords_sample = coords[sample_idx]
moments_sample = moments_final[:, sample_idx].T  # (nsample, 3)

fig, ax = plt.subplots(figsize=(8, 8))
ax.quiver(
    coords_sample[:, 0], coords_sample[:, 1],
    moments_sample[:, 0], moments_sample[:, 1],
    moments_sample[:, 2],  # color by m_z
    cmap='coolwarm', scale=50, width=0.003
)
ax.set_xlabel('x (Å)')
ax.set_ylabel('y (Å)')
ax.set_title(f'Spin Configuration (sample of {len(sample_idx)} atoms)')
ax.set_aspect('equal')
plt.colorbar(ax.collections[0], ax=ax, label='$m_z$')
plt.tight_layout()
plt.show()

## 8. Run Dynamics (Optional)
Evolve the system in time using LLG.

In [ ]:
# Uncomment to run time evolution
# with Simulator() as sim:
#     trajectory = []
#     def record_step(step, moments, energy):
#         if step % 10 == 0:
#             trajectory.append(moments.copy())
#             mag = np.linalg.norm(np.mean(moments, axis=(1,2)))
#             print(f"Step {step}: |M| = {mag:.3f}, E = {energy:.3f}")
#     
#     sim.relax(mode='S', temperature=300, steps=100, 
#               timestep=1e-16, damping=0.05, callback=record_step)
#     
#     print(f"Recorded {len(trajectory)} trajectory snapshots")

## Summary
This notebook demonstrated:
- ✓ Define crystal structure programmatically
- ✓ Set up exchange interactions
- ✓ Run simulations via Python API
- ✓ Analyze and visualize results

**Next steps:**
- Modify lattice/exchange parameters for different materials
- Add anisotropy, external fields, DM interactions
- Compute observables (susceptibility, correlation functions)
- Run temperature scans